# Ejercicio aplicado de DataFrames y Spark SQL

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [5]:
## Importa el csv de "data/WorldCupPlayers.csv"
## Visualiza los datos

wcp = spark.read.csv('C:/Users/abrah/Documents/Primer_semestre_MCD/Bases de datos/Data_Ejercicios/WorldCupPlayers.csv', header=True, inferSchema=True)

wcp.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

In [6]:
## ¿que tipo de datos contiene cada variable?
wcp.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [8]:
## ¿Cuantos registros hay?

registros = wcp.count()
print(f"Número de registros: {registros}")

Número de registros: 37784


In [10]:
## Obtén los principales estadísticos de Position

est_position = wcp.select('Position').describe()

est_position.show()

+-------+--------+
|summary|Position|
+-------+--------+
|  count|    4143|
|   mean|    NULL|
| stddev|    NULL|
|    min|       C|
|    max|     GKC|
+-------+--------+



In [12]:
## Slecciona y muestra los registros distintos de 'Player Name','Coach Name'

dist = wcp.select('Player Name', 'Coach Name').distinct()
dist.show()

+--------------------+--------------------+
|         Player Name|          Coach Name|
+--------------------+--------------------+
|    Arturo FERNANDEZ| BRU Francisco (ESP)|
|Cayetano CARRERAS...|DURAND LAGUNA Jos...|
|  Ernesto MASCHERONI|SUPPICI Alberto (...|
|          Aziz FAHMY|   McREA James (SCO)|
|        Gyula POLGAR|    NADAS Odon (HUN)|
|  Ernesto ALBARRACIN|PASCUCCI Felipe (...|
| Armando CASTELLAZZI|POZZO Vittorio (ITA)|
|     Jaroslav BOUCEK|   PETRU Karel (TCH)|
|           Erwin NYC|  KALUZA Jozef (POL)|
|     Stanislaw BARAN|  KALUZA Jozef (POL)|
|     Fernando ROLDAN|BUCCIARDI Arturo ...|
|            Joe MACA|  JEFFREY Bill (SCO)|
|               INDIO|  MOREIRA Zeze (BRA)|
|      Rene DEREUDDRE|PIBAROT Pierre (FRA)|
|    Anton MALATINSKY|    CEJP Josef (TCH)|
|    Alberto MARIOTTI|LORENZO Juan Carl...|
|  Alfredo DI STEFANO|HERRERA Helenio (...|
|             FIDELIS| FEOLA Vicente (BRA)|
|     Stoyan YORDANOV|BOZHKOV Stefan (BUL)|
|      Wim RIJSBERGEN| MICHELS R

In [15]:
## ¿Cuantos partidos con el ID de 1096 ha habido?

conteo = wcp.filter(wcp['MatchID'] == 1096).count()
print(f"Número de partidos con ID 1096: {conteo}")

Número de partidos con ID 1096: 33


In [19]:
## Muestra los datos donde la posicion haya sido C y el evento sea G40

filtro = wcp.filter((wcp['Position'] == 'C') & (wcp['Event'] == 'G40'))
filtro.show()

+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+



In [23]:
## Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20

from pyspark.sql.functions import expr

wcp.createOrReplaceTempView("MatchID")

cond = """
CASE 
    WHEN MatchID >= 20 THEN 'Cumple'
    ELSE 'No cumple'
END as MatchCondition
"""

# Mostrar los resultados
newdf = wcp.withColumn("MatchID", expr(cond))
newdf.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201| Cumple|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201| Cumple|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201| Cumple|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201| Cumple|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201| Cumple|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201| Cumple|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 